# Dataset, documents, FAISS; retriever

## 🔹 Load the dataset containing the tuples `(query, correct_answer, distractor_1, distractor_2)` and the one containing the documents

In [1]:
from datasets import load_dataset

dataset = load_dataset('saracandu/msmarco_modified', split="train", trust_remote_code=True)
dataset

Dataset({
    features: ['Unnamed: 0', 'answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers', 'correct_answer', 'distractor_1', 'distractor_2'],
    num_rows: 82326
})

In [11]:
dataset[1]['passages']

'{\'is_selected\': [0, 1, 0, 0, 0, 0, 0], \'passage_text\': [\'In his younger years, Ronald Reagan was a member of the Democratic Party and campaigned for Democratic candidates; however, his views grew more conservative over time, and in the early 1960s he officially became a Republican. In November 1984, Ronald Reagan was reelected in a landslide, defeating Walter Mondale and his running mate Geraldine Ferraro (1935-), the first female vice-presidential candidate from a major U.S. political party.\', "From Wikipedia, the free encyclopedia. A Reagan Democrat is a traditionally Democratic voter in the United States, especially a white working-class Northerner, who defected from their party to support Republican President Ronald Reagan in either or both the 1980 and 1984 elections. During the 1980 election a dramatic number of voters in the U.S., disillusioned with the economic \'malaise\' of the 1970s and the presidency of Jimmy Carter (even more than, four years earlier, Liberal Republ

In [23]:
selected_passages = []

for row in dataset:
    passages_data = ast.literal_eval(row['passages'])
    try:
        selected_index = passages_data['is_selected'].index(1)
        selected_passage = {
            'is_selected': 1,
            'passage_text': passages_data['passage_text'][selected_index],
        }
        selected_passages.append(selected_passage)
    except ValueError:
        # Aggiungi un passaggio vuoto se non c'è nessun passaggio selezionato
        selected_passages.append({'is_selected': 0, 'passage_text': ''})

# Assicurati che la lunghezza dei passaggi selezionati corrisponda alla lunghezza del dataset originale
assert len(selected_passages) == len(dataset), "Errore: lunghezza dei passaggi selezionati non corrisponde alla lunghezza del dataset originale"

In [24]:
from datasets import Dataset

# Creazione di un nuovo dizionario con i dati desiderati
new_dataset = {
    'answers': dataset['answers'],
    'query': dataset['query'],
    'query_id': dataset['query_id'],
    'query_type': dataset['query_type'],
    'wellFormedAnswers': dataset['wellFormedAnswers'],
    'correct_answer': dataset['correct_answer'],
    'distractor_1': dataset['distractor_1'],
    'distractor_2': dataset['distractor_2'],
    'selected_passages': selected_passages
}

# Creazione del nuovo dataset
new_dataset = Dataset.from_dict(new_dataset)

In [29]:
new_dataset[20]

{'answers': "['The Camerata are a group of four powerful and influential individuals in the city of Cloudbank. Established by Grant Kendrell and Royce Bracket, the group circumvents the official administration and democratic nature of Cloudbank, in order to establish some form of stability in the ever-changing city.']",
 'query': 'what is a camerata',
 'query_id': 19719,
 'query_type': 'description',
 'wellFormedAnswers': '[]',
 'correct_answer': "['The Camerata are a group of four powerful and influential individuals in the city of Cloudbank. Established by Grant Kendrell and Royce Bracket, the group circumvents the official administration and democratic nature of Cloudbank, in order to establish some form of stability in the ever-changing city.']",
 'distractor_1': "['Stores food (sugar and salts in solution) and water.']",
 'distractor_2': "['Africa']",
 'selected_passages': {'is_selected': 1,
  'passage_text': 'The Camerata are a group of four powerful and influential individuals i

# Model part (`Llama-2-7b-chat-hf`)

## ▪️ Upload the model: 

In [ ]:
# do not run this unless necessary!

from huggingface_hub import login
login()

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

#################################################################
# Tokenizer
#################################################################

model_name="meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 🔹 Pipeline for **thesis** generation

In [2]:
from langchain.llms import HuggingFacePipeline

response_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation", # deve matchare la scheda del modello HF o dà errore
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.5,
    return_full_text=True,
    max_new_tokens=400,
    top_p=0.0
)

response_generation_llm = HuggingFacePipeline(pipeline=response_generation_pipeline)

## ▪️ Select a subset of the true dataset as a test

In [3]:
# select a subset of the queries, just for test:
first_queries = new_dataset['query'][:10]
# first_queries

NameError: name 'new_dataset' is not defined

In [ ]:
# same for correct answers and distractors:
correct_answers = new_dataset['correct_answer'][:10]
distractors_1 = new_dataset['distractor_1'][:10]
distractors_2 = new_dataset['distractor_2'][:10]

In [ ]:
# same for relevant sources
sources = new_dataset['selected_passages'][:10]

## ▪️ Merge the true answer and the distractors into a vector, shuffling the order of the elements

In [ ]:
# shuffles the order of the vector containing the correct answer and the two distractors
# returns another vector, shuffled
import random

def shuffleAnswers(correct_answer, distractor_1, distractor_2):
    merge_options = [correct_answer, distractor_1, distractor_2]
    random.shuffle(merge_options)
    return merge_options

## 🔹 PromptTemplate definition and a LLMChain for the **thesis** 

In [ ]:
# prompt template definition
# requires question, options (a string containing the possible options) and the context as input variables!

from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template(
"""
    You're a helpful assistant and you are asked to answer a question correctly, given a certain number of options. 
    Answer with the correct option only and then stop.
    Given this question: {question} \n
    You must answer by choosing only one option above these: {option_a}, {option_b}, {option_c}. \n
    Here is context to help: {context} \n
    The correct answer is:
 """
)

In [ ]:
# LLM chain definition
from operator import itemgetter

augmentation = {"question": itemgetter("question"),
                "option_a": itemgetter("option_a"), 
                "option_b": itemgetter("option_b"),
                "option_c": itemgetter("option_c"),
                "context": itemgetter("context"), }

thesis_chain = augmentation | prompt_template | response_generation_llm

In [ ]:
prompt_template

In [ ]:
response_generation_llm

In [ ]:
thesis_chain

## 🔹 Function that generates the output given the prompt, the question and the set of options

In [ ]:
def thesisGeneration(query, prompt_template, merged, sources):
    given_answer = thesis_chain.invoke({'question': query, 
                                     'option_a': merged[0], 'option_b': merged[1], 'option_c': merged[2], 
                                     'context': sources})
    return given_answer

In [ ]:
# extract the true answer (i.e. remove the unnecessary)

def extract_answer(text):
    # trova l'indice in cui inizia il testo "the correct answer is:"
    start_index = text.find("The correct answer is:") + len("The correct answer is:")
    # estrai il testo dopo "The correct answer is:"
    correct_answer_text = text[start_index:].strip()
    
    return correct_answer_text

## 🔹 Test: how well the thesis alone is able to perform?

In [ ]:
answers = []
for i in range(10):
    print(f"True answer: {correct_answers[i]}")
    merged_options = shuffleAnswers(correct_answers[i], distractors_1[i], distractors_2[i])
    answers.append(extract_answer(thesisGeneration(first_queries[i], prompt_template, merged_options, sources[i])))
    print(f"Given answer: {extract_answer(thesisGeneration(first_queries[i], prompt_template, merged_options, sources[i]))}")
    print('****************')

In [ ]:
answers

## 🔸 Pipeline for **antithesis** generation

In [48]:
response_check_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.5,
    return_full_text=True,
    max_new_tokens=400,
    top_p=0.0
)

response_check_llm = HuggingFacePipeline(pipeline=response_check_pipeline)

## 🔸 PromptTemplate definition and a LLMChain for the **antithesis** 

In [49]:
from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template(
"""
    You're a helpful assistant and you are asked to check whether or not a question was answered correctly, given a certain number of candidate options and the context. 
    Given this question: {question} \n 
    These are the possible options: {option_a}, {option_b}, {option_c} \n.
    The answer that you have to check is {candidate_answer}. 
    Here is context to help: {context} \n
    Answer by saying 'Yes' if it is correct and 'No' otherwise, then explain why you think so.
    Why or why not the answer is correct:
 """
)

In [50]:
# LLM chain definition

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

augmentation = {"question": itemgetter("question"),
                "option_a": itemgetter("option_a"), 
                "option_b": itemgetter("option_b"),
                "option_c": itemgetter("option_c"),
                "candidate_answer": itemgetter("candidate_answer"),
                "context": itemgetter("context"), }

antithesis_chain = augmentation | prompt_template | response_check_llm

## 🔸 Function to generate the antithesis given the question, the thesis, the context and the options

In [51]:
def antithesisGeneration(query, prompt_template, merged, candidate_answer, sources):
    
    second_answer = antithesis_chain.invoke({'question': query, 
                                            'option_a': merged[0], 'option_b': merged[1], 'option_c': merged[2], 
                                            'candidate_answer': candidate_answer,
                                            'context': sources})
    return second_answer

In [52]:
def extract_answer_ant(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("Why or why not the answer is correct:")
    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("Why or why not the answer is correct:")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [81]:
ant_answers = []
for i in range(50):
    print(f"True answer: {correct_answers[i]}")
    merged_options = shuffleAnswers(correct_answers[i], distractors_1[i], distractors_2[i])
    ant_answers.append(extract_answer_ant(antithesisGeneration(first_queries[i], prompt_template, merged_options, answers[i], sources[i])))
    print(f"Given answer: {extract_answer_ant(antithesisGeneration(first_queries[i], prompt_template, merged_options, answers[i], sources[i]))}")
    print('****************')

True answer: ['Results-Based Accountability is a disciplined way of thinking and taking action that communities can use to improve the lives of children, youth, families, adults and the community as a whole.']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: Yes - This option matches exactly the passage text provided in the prompt. Therefore, based solely upon the information presented herein, we may confidentiality say "yes" because there exists no other possibility more likely than others among these three choices offered above; however please note while checking against actual knowledge about topic might reveal additional relevant details leading us towards another conclusion!
****************
True answer: ['Yes']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: I would say yes because according to what we know about how people voted during those times as described on wikipedia, many Democrats did indeed switch parties for reasons related specifically to presidential candidates like jimmy carter being seen negatively compared to republican counterparts such as gerry ford. This shift away form traditional allegiance towards another political ideology became known colloquilly among some groups within these communities themselves - hence terming them “Reagans”!
****************
True answer: ['20-25 minutes']


...

Given answer: Yes - This option should be selected because in the passage provided there isn’t any mention about $107 being used as an ingredient for killing scorpios naturally nor does anyone use such amount when talking abut natural ways o f getting rid off them. Also,the other two opyons don t make sense either since they aren ot related directly wiht anything mentioned int he pasge
****************
True answer: ['It is the ritual ceremony of encounter.']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: Yes - This option matches both the passage text provided as well as any additional information from within the prompt itself such as the word count for example which states specifically that one possibility among others could fit into an allocated time frame
****************
True answer: ['Often refers to a man who is able to grow a corn-silk like beard.']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: Yes - This option matches all parts ofthe definition providedin thestring. It mentions growinga "corn-silk"like beard which fits with part oneofthedefinitionand also includes thementioningeclectictaste ssuchasmusicandsauthorswhichfitthe thirdpartoftheaddefinition."Aburlywoodisfriend whoreliable".
****************


In [75]:
ant_answers

['Yes - This option matches exactly the passage text provided in terms of wordings match between Options A & B passages respectively; therefore I will say "yes" because they both mean same thing based from how each words were defined within those two paragraph sections!',
 'I would say yes because according to what we know about presidential history as well as political science research on voting behavior during his time period there were indeed many Democrats across America including those living up north particularly among blue collar workers whose views aligned closely enough for them to vote republican under presidents like him even though they may still identify themselves politically leftward overall thus making such individuals known historians today refer too collectively simply put -Reagon Dems',
 'Yes - This option matches exactly what the passage says about taking thetraintoConcordeWeststationonthenorthernline( redlinenotspecial) which takes around twenty five minutefromthec

## 🔺 Pipeline for **synthesis** generation

In [56]:
final_response_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.5,
    return_full_text=True,
    max_new_tokens=400,
    top_p=0.0
)

final_response_llm = HuggingFacePipeline(pipeline=final_response_pipeline)

## 🔺 PromptTemplate definition and a LLMChain for the **synthesis** 

In [57]:
from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template(
"""
    You're a helpful assistant and you are asked to answer a certain question, given a certain number of candidate options and the context.
    You are also provided with an initial response and its critique, that could enforce or not the first opinion.
    Make a reasonable synthesis of these two opinions, but answer by outputting exactly one of the answer options only.
    Given this question: {question} \n 
    These are the possible options: {option_a}, {option_b}, {option_c} \n.
    The answer that you have to check is {candidate_answer} and this is its critique: {critique}.
    Here is context to help: {context} \n
    The answer is:
 """
)

In [58]:
# LLM chain definition

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

augmentation = {"question": itemgetter("question"),
                "option_a": itemgetter("option_a"), 
                "option_b": itemgetter("option_b"),
                "option_c": itemgetter("option_c"),
                "candidate_answer": itemgetter("candidate_answer"),
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }

synthesis_chain = augmentation | prompt_template | response_check_llm

## 🔺 Function to generate the synthesis given literally everything

In [60]:
def synthesisGeneration(query, prompt_template, merged, candidate_answer, critique, sources):
    final_answer = synthesis_chain.invoke({'question': query, 
                                            'option_a': merged[0], 'option_b': merged[1], 'option_c': merged[2], 
                                            'candidate_answer': candidate_answer,
                                            'critique': critique,
                                            'context': sources})
    return final_answer

In [61]:
def extract_answer_syn(text):
    # Trova l'indice in cui inizia il testo "The answer is:"
    start_index = text.find("The answer is:")
    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        # Estrai il testo dopo "The answer is:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [82]:
syn_answers = []
for i in range(50):
    print(f"True answer: {correct_answers[i]}")
    merged_options = shuffleAnswers(correct_answers[i], distractors_1[i], distractors_2[i])
    syn_answers.append(extract_answer_syn(synthesisGeneration(first_queries[i], prompt_template, merged_options, answers[i], ant_answers[i], sources[i])))
    print(f"Given answer: {extract_answer_syn(synthesisGeneration(first_queries[i], prompt_template, merged_options, answers[i], ant_answers[i], sources[i]))}")
    print('****************')

True answer: ['Results-Based Accountability is a disciplined way of thinking and taking action that communities can use to improve the lives of children, youth, families, adults and the community as a whole.']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: The answer is:
  Results-Based Accountability
****************
True answer: ['Yes']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: The answer is:
  No
****************
True answer: ['20-25 minutes']


...

Given answer: The answer is:
  [[$107.  2]]
****************
True answer: ['It is the ritual ceremony of encounter.']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: The answer is:
  It is the ritual ceremony of encounter
****************
True answer: ['Often refers to a man who is able to grow a corn-silk like beard.']


/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pipelines/base.py:1167: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/transformers/pip

Given answer: The answer is:
  ["Often referestoamanthatgrows acorn-silk likebeard"]
****************


In [83]:
syn_answers

['The answer is:\n  Results-Based Accountability',
 'The answer is:\n  No',
 'The answer is:\n  [[20-25 minutes]]',
 "The answer is:\n  [['$11 to $22 per square foot')]]",
 'The answer is:\n  [[ due to Symptons In Body ]].',
 'The answer is:\n  Inthe Rib Cage',
 'The answer is:\n  [[ Internationa lpatens, domestic]]',
 'The answer is:\n  Light coloured Igneous Rocks',
 "The answer is:\n  [[('Aeschlyulus'),'$140-$$202')]]",
 'The answer is:\n  Lapel pin',
 'The answer is:\n \n[B] Gametes produce by Meiotic Division',
 'The answer is:\n  [[Dr.Seuss]]',
 'The answer is:\n  * Four additionalsessionof medianschoolduringthefourthandfiftyearoftrainingtoturnintothespecializationyouwant',
 'The answer is:\n  $$6-$16\\space{\\rm{a}\\ space}$',
 'The answer is:\n  * Granito',
 'The answer is:\n  $$4.64 – $6.36$$',
 'The answer is:\n  [Fish]',
 'The answer is:\n  [[True]].',
 'The answer is:\n  Macintosh',
 'The answer is:\n  [[7 Days Before To 7Days After Rash Appear]]',
 'The answer is:\n  [[Gra

In [84]:
correct_answers

["['Results-Based Accountability is a disciplined way of thinking and taking action that communities can use to improve the lives of children, youth, families, adults and the community as a whole.']",
 "['Yes']",
 "['20-25 minutes']",
 "['$11 to $22 per square foot']",
 "['Due to symptoms in the body']",
 "['Inside the rib cage.']",
 "['The most expensive patents are international patents, which can run up to $100,000 or higher.Domestically the costs can be $10,000 or above.']",
 '[]',
 "['Sophocles, Aeschylus and Euripides']",
 "['A tree or shrub which produces distinctive cones as part of its sexual reproduction.']",
 "['Somatic cells are produced by mitosis and gametes produced by most organisms combine to form a zygote with n pairs of chromosomes.']",
 "['Dr. Seuss']",
 "['45 minutes to an hour']",
 "['$6-$16 a square foot']",
 "['Granite.', 'Granite.']",
 "['$4.64 - $6.36']",
 "['Fish']",
 '["In a New York State hermit\'s letter to the editor of an Adirondack Mountain newspaper."]

In [79]:
answers

["[['Results-Based Accountancy']]",
 "[['No']]",
 "[['20-25 minutes']]",
 "[['$11 to $22 per square foot']]",
 "[[' Due to symtions in the bod']]"]